# P0: COVID-19 Progress to zero metric for the US

P0 metric defined by: 
Juan M. Lavista Ferres MSCa1 , Ruth B Etzioni PhDb2 and William B. Weeks MD, MBA, PhDc1

Data Exploration and Visualization: Adriana Moscatelli

In [1]:
import pandas as pd
import numpy as np

## Data Source: USA Facts covid-19 cases and deaths data

In [2]:
master_data_confirmed = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv')
master_data_deaths = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv')

### Calculation of the latest progress to zero (PO) datapoint.

P0 at time i is defined one minus the ratio of A(i) to B(i), where A is the 7-day
moving average of new cases ending at day i and B is the 7-day moving average of
new cases corresponding to the historical peak. The measure could be defined using
a shorter interval for the moving average, or it could be defined using a weighted
moving average, but to illustrate the idea, we use a simple 7-day moving average.
Because the measure would not be stable with small numbers of cases, we recommend limiting measure application to geographies with at least 100 COVID-19
cases confirmed within a region.
Defining a 7-day moving average of cases and i as the current day, the calculations
are provided below:

\begin{equation*}
[1] ma(j) <- \frac{1}{7}\sum_{k=j-6}^{j} newcases[k]
\end{equation*}

\begin{equation*}
[2] P0=1-(\frac{ma(i)}{max(ma(7),...,ma(i))})
\end{equation*}

In [3]:
# Adriana: This function is used to calculate daily increment values on datasets that present the data by date on each column where the values on each cell are accumulated. Our goal is to get daily increase values. 
def daily_DF(df):
    # Copy the data frame
    dfdaily=df.copy()

    # We are on the first iteration
    first=True
    # For every column
    for column in df:
       # If this is the first iteration
       if first:
         # Save the first column
         prev = column
         # Copy over the first column unchanged
         dfdaily[column]=df[column]
         # All subsequent iterations are not the first
         first=False
         # Skip the rest of the loop
         continue
       # We are not the first iteration, so keep going
       # Set the new column contents to the difference of the
       # previous column and the current column
       dfdaily[column] = df[column]-df[prev]
       # The current column is the previous column for the next iteration
       prev = column
        
    # Convert negatives into zero
    num = dfdaily._get_numeric_data()
    num[num < 0] = 0
    
    return(dfdaily)

In [4]:
# Remove all columns that are not dates so we can apply the daily_DF function to only columns with values for confirmed or deaths
only_dates_confirmed = master_data_confirmed.drop(['countyFIPS', 'County Name', 'State', 'stateFIPS'], axis=1)
only_dates_deaths = master_data_deaths.drop(['countyFIPS', 'County Name', 'State', 'stateFIPS'], axis=1)

In [5]:
# Apply the daily_DF function. We get a dataframe with daily increase values
daily_confirmed = daily_DF(only_dates_confirmed)
daily_deaths = daily_DF(only_dates_deaths)

In [6]:
# Extract some columns from the master data we got from USA Facts. We want to bring those coumns back into our daily increase dataframe.
only_non_dates_confirmed = master_data_confirmed[['County Name', 'State']].copy()
only_non_dates_deaths = master_data_deaths[['County Name', 'State']].copy()

In [7]:
# Concatenate the columns from the original master table with the daily increase values
confirmed_dates_as_columns = pd.concat([only_non_dates_confirmed, daily_confirmed], axis=1)
deaths_dates_as_columns = pd.concat([only_non_dates_deaths, daily_deaths], axis=1)

In [8]:
# Use the melt function to pivot the table (we want dates as columns)
confirmed_melt = confirmed_dates_as_columns.melt(id_vars =['State', 'County Name'])
confirmed = confirmed_melt.rename(columns={"variable": "Date", "value": "Confirmed"})

deaths_melt = deaths_dates_as_columns.melt(id_vars = ['State', 'County Name'])
deaths = deaths_melt.rename(columns={"variable": "Date", "value": "Deaths"})

In [9]:
# Remove the word County from County Name. We don't want it in the visualization
confirmed['County Name'] = confirmed['County Name'].str.replace(r'County', '')
# And change the name of the column for consistency
confirmed = confirmed.rename(columns={"County Name": "County_name"})

In [10]:
# Add/concatenate the Deaths column into the Confirmed table. We only want one table/dataframe
confirmed_and_deaths = pd.concat([confirmed, deaths['Deaths']], axis=1)

In [11]:
confirmed_and_deaths.dtypes

State          object
County_name    object
Date           object
Confirmed       int64
Deaths          int64
dtype: object

In [12]:
# We convert the Date column into Date/Time so we can find the max/latest date. It is also needed for the groupby operation later
import datetime as dt
confirmed_and_deaths['Date'] = confirmed_and_deaths['Date'].apply(lambda x:dt.datetime.strptime(x,'%m/%d/%y'))

In [13]:
# Create a State_County column for operations later so we don't have to groupby two columns
confirmed_and_deaths['State_County'] = confirmed_and_deaths['State'].str.cat(confirmed_and_deaths['County_name'],sep=" ")

In [14]:
# We copy the acronym column to use with Plotly maps
confirmed_and_deaths['State_acronym'] = confirmed_and_deaths['State']

In [15]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [16]:
# We replace the state acronym with the full word to make it look better in the visualization
confirmed_and_deaths['State'] = confirmed_and_deaths['State'].map(states).fillna(confirmed_and_deaths['State'])

In [17]:
# Save to csv
confirmed_and_deaths.to_csv('USAFacts_daily.csv')

In [18]:
# We make a copy of confirmed_and_deaths_counties, which will will use later to crate a dataframe at the state level
confirmed_and_deaths_counties = confirmed_and_deaths.copy()

## Progress to Zero at the County level

In [19]:
# We calculate the 7 day moving average for confirmed and deaths for all counties in all states
confirmed_and_deaths_counties['Confirmed_moving_average'] = confirmed_and_deaths_counties.groupby(['State_County'])['Confirmed'].transform(lambda x: x.rolling(7, 1).mean())
confirmed_and_deaths_counties['Deaths_moving_average'] = confirmed_and_deaths_counties.groupby(['State_County'])['Deaths'].transform(lambda x: x.rolling(7, 1).mean())

In [20]:
# Get the peak value of the moving average at the county level
peak_confirmed_counties = confirmed_and_deaths_counties.groupby(['State_County'])['Confirmed_moving_average'].max()
confirmed_and_deaths_counties['Peak_confirmed'] = confirmed_and_deaths_counties['State_County'].map(peak_confirmed_counties)

# Get the peak value of the moving average at the county level
peak_deaths_counties = confirmed_and_deaths_counties.groupby(['State_County'])['Deaths_moving_average'].max()
confirmed_and_deaths_counties['Peak_deaths'] = confirmed_and_deaths_counties['State_County'].map(peak_deaths_counties)

In [21]:
# Get the latest value
# We find the last date and store the value
#from datetime import datetime, timedelta
#last_obs = confirmed_and_deaths_counties['Date'].max() - timedelta(days=6)
last_obs = confirmed_and_deaths_counties['Date'].max()

last_avg_confirmed = confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties['Date'] == last_obs, 'Confirmed_moving_average']
confirmed_and_deaths_counties['Last_avg_obs_confirmed'] = last_avg_confirmed

last_avg_deaths = confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties['Date'] == last_obs, 'Deaths_moving_average']
confirmed_and_deaths_counties['Last_avg_obs_deaths'] = last_avg_deaths

In [22]:
# Compute the Progress to Zero metric for confirmed cases and deaths averages and the peak for counties
confirmed_and_deaths_counties['Progress_to_zero_confirmed'] = 1 - (confirmed_and_deaths_counties['Confirmed_moving_average']/confirmed_and_deaths_counties['Peak_confirmed'])
confirmed_and_deaths_counties['Progress_to_zero_deaths'] = 1 - (confirmed_and_deaths_counties['Deaths_moving_average']/confirmed_and_deaths_counties['Peak_deaths'])

In [23]:
# Cleaning values. We only want to see Progress to Zero for the last observation
confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties.Last_avg_obs_confirmed.isnull(), 'Peak_confirmed'] = confirmed_and_deaths_counties.Last_avg_obs_confirmed

confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties.Last_avg_obs_deaths.isnull(), 'Peak_deaths'] = confirmed_and_deaths_counties.Last_avg_obs_deaths

confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties.Last_avg_obs_confirmed.isnull(), 'Progress_to_zero_confirmed'] = confirmed_and_deaths_counties.Last_avg_obs_confirmed

confirmed_and_deaths_counties.loc[confirmed_and_deaths_counties.Last_avg_obs_deaths.isnull(), 'Progress_to_zero_deaths'] = confirmed_and_deaths_counties.Last_avg_obs_deaths

In [24]:
# Create a geo-level column. Used for visualization in PowerBI
confirmed_and_deaths_counties["geo_level"] = 'County'

In [25]:
# First we round to 2 decimal points
confirmed_and_deaths_counties = confirmed_and_deaths_counties.round(2)
confirmed_and_deaths_counties.to_csv('progress_to_zero_metric_US_counties.csv')

## Progress to Zero at the State level

In [26]:
# We create a dataframe at the State level
confirmed_and_deaths_states = confirmed_and_deaths.groupby(['State', 'State_acronym', 'Date'])['Confirmed', 'Deaths'].sum()
confirmed_and_deaths_states = confirmed_and_deaths_states.reset_index()

In [27]:
# We calculate the 7 day moving average for confirmed and deaths for all states
confirmed_and_deaths_states['Confirmed_moving_average'] = confirmed_and_deaths_states.groupby('State')['Confirmed'].transform(lambda x: x.rolling(7, 1).mean())
confirmed_and_deaths_states['Deaths_moving_average'] = confirmed_and_deaths_states.groupby('State')['Deaths'].transform(lambda x: x.rolling(7, 1).mean())

In [28]:
# Get the peak value of the moving average at the State level
peak_confirmed_states = confirmed_and_deaths_states.groupby('State')['Confirmed_moving_average'].max()
confirmed_and_deaths_states['Peak_confirmed'] = confirmed_and_deaths_states['State'].map(peak_confirmed_states)

peak_deaths_states = confirmed_and_deaths_states.groupby('State')['Deaths_moving_average'].max()
confirmed_and_deaths_states['Peak_deaths'] = confirmed_and_deaths_states['State'].map(peak_deaths_states)

In [29]:
# Get the latest value
# We find the last date and store the value
from datetime import datetime, timedelta
last_obs_states = confirmed_and_deaths_states['Date'].max()

last_avg_confirmed_states = confirmed_and_deaths_states.loc[confirmed_and_deaths_states['Date'] == last_obs_states, 'Confirmed_moving_average']
confirmed_and_deaths_states['Last_avg_obs_confirmed'] = last_avg_confirmed_states

last_avg_deaths_states = confirmed_and_deaths_states.loc[confirmed_and_deaths_states['Date'] == last_obs_states, 'Deaths_moving_average']
confirmed_and_deaths_states['Last_avg_obs_deaths'] = last_avg_deaths_states

In [30]:
# Compute the Progress to Zero metric for confirmed cases and deaths averages and the peak for States
confirmed_and_deaths_states['Progress_to_zero_confirmed'] = 1 - (confirmed_and_deaths_states['Confirmed_moving_average']/confirmed_and_deaths_states['Peak_confirmed'])
confirmed_and_deaths_states['Progress_to_zero_deaths'] = 1 - (confirmed_and_deaths_states['Deaths_moving_average']/confirmed_and_deaths_states['Peak_deaths'])

In [31]:
# Cleaning values. We only want to see Progress to Zero for the last observation
confirmed_and_deaths_states.loc[confirmed_and_deaths_states.Last_avg_obs_confirmed.isnull(), 'Peak_confirmed'] = confirmed_and_deaths_states.Last_avg_obs_confirmed

confirmed_and_deaths_states.loc[confirmed_and_deaths_states.Last_avg_obs_deaths.isnull(), 'Peak_deaths'] = confirmed_and_deaths_states.Last_avg_obs_deaths

confirmed_and_deaths_states.loc[confirmed_and_deaths_states.Last_avg_obs_confirmed.isnull(), 'Progress_to_zero_confirmed'] = confirmed_and_deaths_states.Last_avg_obs_confirmed

confirmed_and_deaths_states.loc[confirmed_and_deaths_states.Last_avg_obs_deaths.isnull(), 'Progress_to_zero_deaths'] = confirmed_and_deaths_states.Last_avg_obs_deaths

In [32]:
# Create a geo-level column. Used for visualization in PowerBI
confirmed_and_deaths_states["geo_level"] = 'State'

In [33]:
# Create a column named County_name so we can merge this table with the counties table for visualization in PowerBI (to use the hierarchy in the slicer UI widget)
confirmed_and_deaths_states["County_name"] = confirmed_and_deaths_states["State"]

# Convert to all caps
confirmed_and_deaths_states["County_name"] = confirmed_and_deaths_states["County_name"].str.upper()

# Add word STATE
confirmed_and_deaths_states["County_name"] = confirmed_and_deaths_states["County_name"] + ' STATE'
confirmed_and_deaths_states["County_name"] = '_' + confirmed_and_deaths_states["County_name"].astype(str)

In [34]:
# First we round to 2 decimal points
confirmed_and_deaths_states = confirmed_and_deaths_states.round(2)
confirmed_and_deaths_states.to_csv('progress_to_zero_metric_US_states.csv')

## Progress to Zero US (Counties and States in one table)

In [35]:
confirmed_and_deaths_US = pd.concat([confirmed_and_deaths_states, confirmed_and_deaths_counties])

## Visualizations

In [36]:
# import plotly.graph_objects as go

# fig = go.Figure()

# fig.add_trace(go.Scatter(
#                     x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'WA']['Date'], 
#                     y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'WA']['Confirmed_moving_average'],
#                     mode='lines',
#                     name='USA Facts',
#                     line=dict(color='#0061e6', width=2),
#                     showlegend=True,
#                     hovertemplate = None
#                         )
#              )

# fig.add_trace(go.Scatter(
#                     x=average_for_plot_WA['Date'], 
#                     y=average_for_plot_WA['Confirmed_moving_average'],
#                     mode='lines',
#                     name='DOH',
#                     line=dict(color='#e60000', width=2),
#                     showlegend=True,
#                     hovertemplate = None
#                         )
#              )


# fig.update_layout(title='COVID19 confirmed cases 7 day moving average in WA',
#                   hovermode='x unified')

# fig.show()

# import plotly.io as pio
# pio.write_html(fig, file='WA.html', auto_open=True)

In [37]:
import plotly.express as px
fig = px.bar(confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington'], x='Date', y='Confirmed')
fig.show()

In [38]:
import plotly.express as px

fig = px.line(confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington'], x="Date", y="Confirmed_moving_average", title='Average Confirmed Cases in WA')
fig.show()

In [39]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=3,
                    subplot_titles=("New York", "New Jersey", "Massachusetts",
                                    "Washington", "California", "Washington DC"
                                    ),
                    shared_yaxes=True,
                    print_grid=True,
                    vertical_spacing=0.1,
                    horizontal_spacing=0.06
                   )

fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New York']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New York']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=1, col=1
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New York']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New York']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=1, col=1
             )

fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New Jersey']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New Jersey']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=1, col=2
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New Jersey']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'New Jersey']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=1, col=2
             )


fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Massachusetts']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Massachusetts']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=1, col=3
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Massachusetts']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Massachusetts']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=1, col=3
             )


fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=2, col=1
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'Washington']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=2, col=1
             )



fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'California']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'California']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=2, col=2
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'California']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'California']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=2, col=2
             )

fig.add_trace(go.Bar(
              x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'District of Columbia']['Date'], 
              y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'District of Columbia']['Confirmed'],
              name='Confirmed ',
              textposition='outside',
              hoverlabel=dict(bgcolor='#ff8000', bordercolor='white',
                font=dict(family='Arial', size=20, color='white')
                             ),
              hovertemplate = 'Confirmed: <b>%{y:,.0f}</b><extra></extra>',
              marker_color='#ffa64d',
              marker_line_color = 'white',
              marker_line_width = 0.5,
              showlegend=False
                    ),
              row=2, col=3
             )

fig.add_trace(go.Scatter(
                    x=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'District of Columbia']['Date'], 
                    y=confirmed_and_deaths_states.loc[confirmed_and_deaths_states['State'] == 'District of Columbia']['Confirmed_moving_average'],
                    mode='lines',
                    fill='tozeroy',
                    name='',
                    line=dict(color=' #e67300', width=2),
                    showlegend=False,
                    hoverinfo='skip'
                        ),
              row=2, col=3
             )

# PTZ_NY = confirmed_and_deaths_states.at[4234,'Progress_to_zero_confirmed']*100

# fig.add_annotation(
#         x=confirmed_and_deaths_states.at[4192,'Date'],
#         y=confirmed_and_deaths_states.at[4192,'Confirmed_moving_average'],
#         xref="x",
#         yref="y",
#         text='Progress to zero: 81%',
#         showarrow=True,
#         font=dict(
#             family="Arial", 
#             size=11,
#             color="black"
#             ),  
#         align="center",
#         #arrowhead=2,
#         arrowsize=1,
#         arrowwidth=3,
#         arrowcolor="#9ae5e5",
#         ax=60,
#         ay=-80,
#         axref='pixel',
#         ayref='pixel',
#         #bordercolor="#808080",
#         borderwidth=1,
#         borderpad=14,
#         opacity=0.7,
#         bgcolor='#9ae5e5'
#         )


fig.update_layout(title='States daily confirmed cases',
                  title_x=0.5,
                  barmode='group',
                  paper_bgcolor='rgba(0,0,0,0)', 
                  plot_bgcolor='rgba(0,0,0,0)',
                  height=800,
                  width=1400,
                  uniformtext_minsize=7, uniformtext_mode='hide',
                  #hovermode='x', #sets the bar chart to display both tooltips by default
                  hovermode="x unified",
                  hoverlabel=dict(
                                bgcolor="white", 
                                font_size=16, 
                                font_family="Arial",
                                bordercolor='#b3b3b3'),
                  legend=dict(x=0, y=1),
                  font=dict(size = 12),
                  dragmode=False, #disable panning so mobile apps can receive pinch to zoom events
                #   yaxis = dict(
                #       scaleanchor = "x",
                #       scaleratio = 1,
                #               )
                 )
                 
                     
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='#a6a6a6', color='#4d4d4d',
                #showgrid=True, gridwidth=1, gridcolor='#f2f2f2',
                fixedrange=True,
                #tickfont = dict(size = 10),
                #tickmode='auto', 
                #nticks=3,
                tickmode = 'array',
                #tickvals = [NY.index.min(), NY.index.max()], #gets the first and last date
                #ticktext = ['        Jan 22', 'May 15        '], #manually sets the text
#                 tickmode = 'linear',
#                 tick0 = NY.Date[0],
#                 dtick = 86400000.0 * 15
                spikethickness=0.4,
                spikedash='solid',
                )
              
fig.update_yaxes(showline=True, linewidth=1, linecolor='#a6a6a6', color='#4d4d4d',
                 showgrid=True, gridwidth=1, gridcolor='#f2f2f2',
                 fixedrange=True,
                 tickmode = 'array',
                 tickfont = dict(size = 14),
                 range=(0, 13300)
                )
# fig.update_yaxes(range=[0, 13300], row=1, col=2)
# fig.update_yaxes(range=[0, 8000], row=1, col=2)
# fig.update_yaxes(range=[0, 8000], row=1, col=3)
# fig.update_yaxes(range=[0, 2600], row=2, col=1)
# fig.update_yaxes(range=[0, 2600], row=2, col=2)
# fig.update_yaxes(range=[0, 2600], row=2, col=3)

#fig.update_traces(texttemplate='%{y:,.2s}', textposition='outside')



fig.show(displayModeBar=False)

import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)

import os

if not os.path.exists("images"):
    os.mkdir("images")
    
fig.write_image("images/daily_cases.png")

This is the format of your plot grid:
[ (1,1) x,y   ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]



In [40]:
confirmed_and_deaths_counties['Date'].isnull().sum()

0

In [41]:
confirmed_and_deaths_counties.dtypes

State                                 object
County_name                           object
Date                          datetime64[ns]
Confirmed                              int64
Deaths                                 int64
State_County                          object
State_acronym                         object
Confirmed_moving_average             float64
Deaths_moving_average                float64
Peak_confirmed                       float64
Peak_deaths                          float64
Last_avg_obs_confirmed               float64
Last_avg_obs_deaths                  float64
Progress_to_zero_confirmed           float64
Progress_to_zero_deaths              float64
geo_level                             object
dtype: object

In [42]:
import plotly.graph_objects as go
import numpy as np

loc = confirmed_and_deaths_states["State"]

fig = go.Figure(data=go.Choropleth(
    locations=confirmed_and_deaths_states['State_acronym'],
    z=confirmed_and_deaths_states['Progress_to_zero_confirmed'],
    locationmode='USA-states',
    colorscale='matter',
    reversescale=True,
    autocolorscale=False,
    marker_line_color='#ebccd6',
    colorbar = dict(title='Percentage',
                    tickformat = ',.0%'),
    hoverlabel = dict(font=dict(size=16)),
    showlegend = True,
    text = loc,
    hovertemplate = ' %{text} P0 = <b>%{z: ,.0%}</b><extra></extra>',
))

fig.update_layout(
    title_text='<a href = https://www.medrxiv.org/content/10.1101/2020.05.21.20109298/><b>P0: Progress to zero</b> </a><br><i>A simple metric to measure COVID-19 progress by country/region</i><br>P0 corresponds to the percentage decline from a previously recorded peak level.<br> The metric ranges from 0% (representing a geography that has not yet peaked) to 100% <br>(representing a geography wherein 0 cases have been recorded for at least seven days).',
    font = dict(size=10),
    margin=dict(l=20, r=20, t=160, b=20),
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'),
    showlegend=False,
    annotations = [dict(xref='paper',
                        yref='paper',
                        x=0, y=-0.02,
                        showarrow=False,
                        text ='Data: USA Facts - Updated: '+ str(last_obs_states.strftime('%Y-%d-%m')))]
)


fig.show(displayModeBar=False)

import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)

import os

if not os.path.exists("images"):
    os.mkdir("images")
    
fig.write_image("images/PTZ_US_map.png")